In [957]:
import os
import numpy as np
import pickle
from sklearn.svm import SVC

from sknetwork.topology import get_connected_components
from utils import *

In [958]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def create_edgeNode_relation(num_nodes, self_loops=False):
    if self_loops:
        indices = np.ones([num_nodes, num_nodes])
    else:
        indices = np.ones([num_nodes, num_nodes]) - np.eye(num_nodes)
    rel_rec = np.array(encode_onehot(np.where(indices)[0]), dtype=np.float32)
    rel_send = np.array(encode_onehot(np.where(indices)[1]), dtype=np.float32)
    
    return rel_rec, rel_send 

# Pedestrian Data

In [959]:
!ls data/pedestrian/

ETH        Hotel      all        students03 zara01     zara02


## Load Data

In [847]:
!ls data/pedestrian/ETH/split11

ls: data/pedestrian/ETH/split11: No such file or directory


In [1125]:
folder = "Hotel/split21"
path = os.path.join("data/pedestrian/", folder)
print("Path: ",path)

#load training examples
with open(os.path.join(path, "examples_train_unnormalized.pkl"),'rb') as f:
    examples_train = pickle.load(f)
#load training velocities
with open(os.path.join(path, "velocities_train.pkl"), 'rb') as f:    
    velocities_train = pickle.load(f)
#load training labels
with open(os.path.join(path, "labels_train_numpy.pkl"), 'rb') as f:
    labels_train = pickle.load(f)
    
#load test examples
with open(os.path.join(path, "examples_test_unnormalized.pkl"),'rb') as f:
    examples_test = pickle.load(f)
#load test examples
with open(os.path.join(path, "velocities_test.pkl"), 'rb') as f:
    velocities_test = pickle.load(f)

#load test labels
with open(os.path.join(path, "labels_test_numpy.pkl"),'rb') as f:
    labels_test = pickle.load(f)

Path:  data/pedestrian/Hotel/split21


In [1107]:
len(examples_train)

141

In [1079]:
len(velocities_train)

160

In [1080]:
examples_train[55].shape

(2, 15, 2)

In [1081]:
velocities_train[55].shape

(2, 15, 2)

In [1082]:
examples_train[10].shape

(2, 15, 2)

In [1083]:
labels_train[10].shape

(2,)

## Feature Engineering on Training Data

### Process training labels

In [1126]:
#labels
labels_train = np.concatenate(labels_train)
labels_train[labels_train==0]=-1
print("Labels shape: ", labels_train.shape)

Labels shape:  (2576,)


In [1127]:
labels_train

array([-1, -1, -1, ..., -1, -1, -1])

### Distances Histogram

In [1128]:
hist_dist_train = []

distance_train_max = -np.inf
distance_train_min = np.inf


for example in examples_train:
    #example: shape: [n_atoms, n_timesteps, 2]
    #example = examples_train[0]
    #label = labels_train[0]
    n_atoms = example.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    example_re = example.reshape((example.shape[0], -1))
    senders = np.matmul(rel_send, example_re)
    #shape: [n_edges, n_timesteps*2]
    receivers = np.matmul(rel_rec, example_re)
    #shape: [n_edges, n_timesteps*2]
    
    senders = senders.reshape((senders.shape[0], example.shape[1], example.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers = receivers.reshape((receivers.shape[0], example.shape[1], example.shape[2]))
    #shape: [n_edges, n_timesteps, 2]

    #compute normalized distance histogram
    distance_example = senders-receivers
    distance_example = distance_example**2
    distance_example = distance_example.sum(-1)
    distance_example = np.sqrt(distance_example)

    distance_example_max = distance_example.max()
    distance_example_min = distance_example.min()
    
    #compute distance train max/min
    if distance_train_max < distance_example_max:
        distance_train_max = distance_example_max
    if distance_train_min > distance_example_min:
        distance_train_min = distance_example_min

for example in examples_train:

    #example = examples_train[0]
    #label = labels_train[0]
    n_atoms = example.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    example_re = example.reshape((example.shape[0], -1))
    #shape: [n_atoms, n_timesteps*2]
    senders = np.matmul(rel_send, example_re)
    #shape: [n_edges, n_timesteps*2]
    receivers = np.matmul(rel_rec, example_re)
    #shape: [n_edges, n_timesteps*2]

    senders = senders.reshape((senders.shape[0], example.shape[1], example.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers = receivers.reshape((receivers.shape[0], example.shape[1], example.shape[2]))
    #shape: [n_edges, n_timesteps, 2]

    #compute normalized distance histogram
    distance_example = senders-receivers
    distance_example = distance_example**2
    distance_example = distance_example.sum(-1)
    distance_example = np.sqrt(distance_example)

    #distance_example_max = distance_example.max()
    #distance_example_min = distance_example.min()
    
    bins = np.arange(distance_train_min, distance_train_max, 4)

    hist_dist_example = []
    for d in distance_example:
        if np.histogram(d, bins=bins)[0].sum() > 0:
            hist_d = np.histogram(d, bins=bins)[0]/np.histogram(d,bins=bins)[0].sum()
        else:
            hist_d = np.histogram(d, bins=bins)[0]
        hist_dist_example.append(hist_d)
    
    hist_dist_train = hist_dist_train+hist_dist_example 
    
hist_dist_train = np.array(hist_dist_train)




In [1129]:
hist_dist_train.shape

(2576, 3)

In [1130]:
hist_dist_train

array([[1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       ...,
       [0.46666667, 0.4       , 0.13333333],
       [0.4       , 0.53333333, 0.06666667],
       [0.46666667, 0.4       , 0.13333333]])

### Normalized Histogram of Speed Difference

In [1131]:
hist_speedDiff_train = []

speedDiff_train_max = -np.inf
speedDiff_train_min = np.inf

for velocity in velocities_train:
    #velocity shape: [n_atoms, n_timesteps, 2]
    n_atoms = velocity.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    velocity_re = velocity.reshape((velocity.shape[0], -1))
    #shape: [n_atoms, n_timesteps*2]
    
    senders = np.matmul(rel_send, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    receivers = np.matmul(rel_rec, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    
    senders = senders.reshape((senders.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers = receivers.reshape((receivers.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    
    speed_senders = np.sqrt((senders**2).sum(-1)) #shape: [n_edges, n_timesteps]
    speed_receivers = np.sqrt((receivers**2).sum(-1)) #shape: [n_edges, n_timesteps]
    
    diff_speed = np.abs(speed_receivers-speed_senders) #shape: [n_edges, n_timesteps]
    
    speedDiff_velocity_max = diff_speed.max()
    speedDiff_velocity_min = diff_speed.min()
    
    if speedDiff_train_max < speedDiff_velocity_max:
        speedDiff_train_max = speedDiff_velocity_max
    if speedDiff_train_min > speedDiff_velocity_min:
        speedDiff_train_min = speedDiff_velocity_min
        
for velocity in velocities_train:
    #velocity shape: [n_atoms, n_timesteps, 2]
    n_atoms = velocity.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    velocity_re = velocity.reshape((velocity.shape[0], -1))
    #shape: [n_atoms, n_timesteps*2]
    
    senders = np.matmul(rel_send, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    receivers = np.matmul(rel_rec, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    
    senders = senders.reshape((senders.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers = receivers.reshape((receivers.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    
    speed_senders = np.sqrt((senders**2).sum(-1))
    speed_receivers = np.sqrt((receivers**2).sum(-1)) #shape: [n_edges, n_timesteps]
    
    diff_speed = np.abs(speed_receivers-speed_senders) #shape: [n_edges, n_timeteps]
    
    bins = np.arange(speedDiff_train_min, speedDiff_train_max, 0.5)
    
    hist_diff_speed = []
    for edge in diff_speed:
        if np.histogram(edge, bins=bins)[0].sum() > 0:
            hist = np.histogram(edge, bins=bins)[0]/np.histogram(edge,bins=bins)[0].sum()
        else:
            hist = np.histogram(edge, bins=bins)[0]
        hist_diff_speed.append(hist)
    hist_speedDiff_train = hist_speedDiff_train+hist_diff_speed
hist_speedDiff_train = np.array(hist_speedDiff_train)




In [1090]:
hist_speedDiff_train.shape

(3036, 4)

In [1091]:
hist_speedDiff_train

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [864]:
speedDiff_train_max

2.0654914326077076

In [865]:
speedDiff_train_min

0.0

In [866]:
diff_speed.shape

(12, 15)

In [867]:
bins

array([0. , 0.5, 1. , 1.5, 2. ])

In [868]:
velocities_train[0].shape

(5, 15, 2)

In [869]:
examples_train[0].shape

(5, 15, 2)

### Normalized Histogram of Absolute Difference in direction

In [1132]:
hist_direcDiff_train = []

directDiff_train_max = np.pi+0.25*np.pi
directDiff_train_min = 0

for velocity in velocities_train:
    #velocity shape: [n_atoms, n_timesteps, 2]
    n_atoms = velocity.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    velocity_re = velocity.reshape((velocity.shape[0], -1))
    #shape: [n_atoms, n_timesteps*2]
    
    senders = np.matmul(rel_send, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    receivers = np.matmul(rel_rec, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    
    senders = senders.reshape((senders.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers = receivers.reshape((receivers.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    
    tans_senders = senders[:,:,1]/senders[:,:,0]
    direc_senders = np.arctan(tans_senders)
    tans_receivers = receivers[:,:,1]/receivers[:,:,0]
    direc_receivers = np.arctan(tans_receivers)
    
    diff_direc = np.abs(direc_senders-direc_receivers)
    
    bins = np.arange(directDiff_train_min, directDiff_train_max, 0.25*np.pi)
    
    hist_diff_direc = []
    for edge in diff_direc:
        if np.histogram(edge, bins=bins)[0].sum() > 0:
            hist = np.histogram(edge, bins=bins)[0]/np.histogram(edge,bins=bins)[0].sum()
        else:
            hist = np.histogram(edge, bins=bins)[0]
        hist_diff_direc.append(hist)
    hist_direcDiff_train = hist_direcDiff_train+hist_diff_direc
    
hist_direcDiff_train = np.array(hist_direcDiff_train)




/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/36843695.py:23: RuntimeWarning: invalid value encountered in true_divide
  tans_senders = senders[:,:,1]/senders[:,:,0]
/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/36843695.py:25: RuntimeWarning: invalid value encountered in true_divide
  tans_receivers = receivers[:,:,1]/receivers[:,:,0]


In [1017]:
diff_direc.shape

(2, 15)

In [1018]:
hist_direcDiff_train.shape

(1604, 4)

### Normalized Histogram of Absolute Difference in velocity direction and relative position

In [1133]:
hist_diffPV_train = []

for i in range(len(examples_train)):
    location = examples_train[i]
    velocity = velocities_train[i]
    
    n_atoms = location.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    location_re = location.reshape((location.shape[0], -1))
    #shape: [n_atoms, n_timesteps*2]
    velocity_re = velocity.reshape((velocity.shape[0], -1))
    #shape: [n_atoms, n_timesteps*2]
    
    senders_vel = np.matmul(rel_send, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    receivers_vel = np.matmul(rel_rec, velocity_re)
    #shape: [n_edges, n_timesteps*2]
    senders_loc = np.matmul(rel_send, location_re)
    #shape: [n_edges, n_timesteps*2]
    receivers_loc = np.matmul(rel_rec, location_re)
    #shape: [n_edges, n_timesteps*2]
    
    senders_vel = senders_vel.reshape(senders_vel.shape[0], velocity.shape[1], velocity.shape[2])
    receivers_vel = receivers_vel.reshape(receivers_vel.shape[0], velocity.shape[1], velocity.shape[2])
    #shape: [n_edges, n_timesteps, 2]

    senders_loc = senders_loc.reshape(senders_loc.shape[0], location.shape[1], location.shape[2])
    receivers_loc = receivers_loc.reshape(receivers_loc.shape[0], location.shape[1], location.shape[2])
    #shape: [n_edges, n_timesteps, 2]
    
    diff_locations = senders_loc-receivers_loc #shape: [n_edges, n_timesteps, 2]
    diff_locations_tans = diff_locations[:,:,1]/diff_locations[:,:,0] #shape: [n_edges, n_timesteps]
    relative_positions = np.arctan(diff_locations_tans) #shape: [n_edges, n_timesteps]
    
    diff_velocities = senders_vel-receivers_vel #shape: [n_edges, n_timesteps, 2]
    diff_velocities_tans = diff_velocities[:,:,1]/diff_velocities[:,:,0] #shape: [n_edges, n_timesteps]
    relative_vel = np.arctan(diff_velocities_tans) #shape: [n_edges, n_timesteps]
    
    diff_vel_loc = np.abs(relative_positions-relative_vel) #shape: [n_edges, n_timesteps]
    
    bins = np.arange(0, np.pi+0.25*np.pi,0.25*np.pi)
    
    hist_diffPV = []
    for edge in diff_vel_loc:
        if np.histogram(edge, bins=bins)[0].sum() > 0:
            hist = np.histogram(edge, bins=bins)[0]/np.histogram(edge,bins=bins)[0].sum()
        else:
            hist = np.histogram(edge, bins=bins)[0]
        hist_diffPV.append(hist)
        
    hist_diffPV_train = hist_diffPV_train+hist_diffPV
    
hist_diffPV_train = np.array(hist_diffPV_train)





/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/764484398.py:36: RuntimeWarning: invalid value encountered in true_divide
  diff_velocities_tans = diff_velocities[:,:,1]/diff_velocities[:,:,0] #shape: [n_edges, n_timesteps]


In [1020]:
hist_diffPV_train.shape

(1604, 4)

In [1021]:
diff_vel_loc.shape

(2, 15)

In [876]:
len(velocities_train)

191

In [877]:
len(examples_train)

191

In [878]:
location = examples_train[0]

In [879]:
location.shape

(5, 15, 2)

### Building Features

In [1134]:
hist_feat = np.concatenate([hist_dist_train, hist_speedDiff_train, hist_direcDiff_train, hist_diffPV_train], 
                           axis=-1)

In [1135]:
hist_feat.shape

(2576, 15)

## Train model on Training Dataset

In [1136]:
#training model
clf = SVC(gamma="auto")
clf.fit(hist_feat, labels_train)

SVC(gamma='auto')

## Test Model on Test Dataset

In [1116]:
examples_test

[array([[[ 0.95095642,  1.6634628 ],
         [ 0.96252612,  1.2017743 ],
         [ 0.95318673,  0.70313522],
         [ 0.99986335,  0.19581587],
         [ 0.99053659, -0.30331894],
         [ 0.96251712, -0.80026948],
         [ 1.045449  , -1.3325564 ],
         [ 1.0163402 , -1.8500398 ],
         [ 1.0070102 , -2.3504903 ],
         [ 1.0328141 , -2.8955935 ],
         [ 0.94640522, -3.4283172 ],
         [ 0.97223831, -3.9741652 ],
         [ 0.99695155, -4.5400859 ],
         [ 0.94788155, -5.079028  ],
         [ 0.99247352, -5.6271434 ]],
 
        [[ 1.6402916 ,  1.9480051 ],
         [ 1.6312841 ,  1.453364  ],
         [ 1.6222696 ,  0.95833968],
         [ 1.6121501 ,  0.44321909],
         [ 1.6399785 , -0.05798632],
         [ 1.5940849 , -0.54878762],
         [ 1.5839417 , -1.0651391 ],
         [ 1.6486869 , -1.5717751 ],
         [ 1.6570068 , -2.0908805 ],
         [ 1.7769386 , -2.6037024 ],
         [ 1.8382638 , -3.1685871 ],
         [ 1.8271375 , -3.7053197 ]

In [1117]:
example = examples_test[0]
label = labels_test[0]

In [947]:
example.shape

(5, 15, 2)

In [948]:
label.shape

(20,)

In [1137]:
n_atoms = example.shape[0]
rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
example_re = example.reshape((example.shape[0], -1))
senders = np.matmul(rel_send, example_re)
receivers = np.matmul(rel_rec, example_re)

senders = senders.reshape((senders.shape[0], example.shape[1], example.shape[2]))
receivers = receivers.reshape((receivers.shape[0], example.shape[1], example.shape[2]))

#compute normalized distance histogram
distance_example = senders-receivers
distance_example = distance_example**2
distance_example = distance_example.sum(-1)
distance_example = np.sqrt(distance_example)

bins = np.arange(distance_train_min, distance_train_max, 4)

hist_dist_example = []
for d in distance_example:
        if np.histogram(d, bins=bins)[0].sum() > 0:
            hist_d = np.histogram(d, bins=bins)[0]/np.histogram(d,bins=bins)[0].sum()
        else:
            hist_d = np.histogram(d, bins=bins)[0]
        hist_dist_example.append(hist_d)

In [115]:
#predicted_edges = clf.predict(hist_dist_example)
#predicted_edges[predicted_edges==-1]=0

In [116]:
#predicted_edges

In [117]:
#predicted_edges_diag = np.diag(predicted_edges)
#predicted_relation = np.matmul(rel_send.T, np.matmul(predicted_edges_diag, rel_rec))

#print("predicted relations:  ",predicted_relation)
#pred_con = get_connected_components(predicted_relation)


#label_edges_diag = np.diag(label)
#label_relation = np.matmul(rel_send.T, np.matmul(label_edges_diag, rel_rec))

#print("label: ", label_relation)
#label_con = get_connected_components(label_relation)

#recall, precision, F1 = compute_groupMitre_labels(label_con, pred_con)

#print("recall: ", recall)
#print("precision: ", precision)

In [118]:
#predicted_edges_diag.shape

In [119]:
#label

In [120]:
#predicted_edges_diag = []
#predicted_edges = predicted_edges.reshape((-1, n_atoms*(n_atoms-1)))
#for edge in predicted_edges:
#    edge_diag = np.diag(edge)
#    predicted_edges_diag.append(edge_diag)

#predicted_edges_diag = np.array(predicted_edges_diag)
#predicted_relations = np.matmul(rel_send.T, np.matmul(predicted_edges_diag, rel_rec))



#label_edges_diag = []
#label = label.reshape((-1, n_atoms*(n_atoms-1)))
#for edge in label:
#    edge_diag = np.diag(edge)
#    label_edges_diag.append(edge_diag)

#label_edges_diag = np.array(label_edges_diag)
#label_relations = np.matmul(rel_send.T, np.matmul(label_edges_diag, rel_rec))

In [121]:
#predicted_relations

In [122]:
#label_relations

In [123]:
#predicted_edges.shape

### Compute Average Recall, Precision, F1

In [1138]:
recall_test = []
precision_test = []
F1_test = []
predicted_edges_all = []

for i in range(len(examples_test)):
    example = examples_test[i]
    #locations, shape: [n_atoms, n_timesteps, 2]
    velocity = velocities_test[i]
    #velocity, shape: [n_atoms, n_timesteps, 2]
    label = labels_test[i]
    n_atoms = example.shape[0]
    rel_rec, rel_send = create_edgeNode_relation(n_atoms, self_loops=False)
    
    example_re = example.reshape((example.shape[0], -1)) #shape: [n_atoms, n_timesteps*2]
    senders_loc = np.matmul(rel_send, example_re) #shape: [n_edges, n_timesteps*2]
    receivers_loc = np.matmul(rel_rec, example_re) #shape: [n_edges, n_timesteps*2]

    senders_loc = senders_loc.reshape((senders_loc.shape[0], example.shape[1], example.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers_loc = receivers_loc.reshape((receivers_loc.shape[0], example.shape[1], example.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    
    velocity_re = velocity.reshape((velocity.shape[0], -1)) #shape: [n_atoms, n_timesteps*2]
    senders_vel = np.matmul(rel_send, velocity_re) #shape: [n_edges, n_timesteps*2]
    receivers_vel = np.matmul(rel_rec, velocity_re) #shape: [n_edges, n_timesteps*2]
    
    senders_vel = senders_vel.reshape((senders_vel.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]
    receivers_vel = receivers_vel.reshape((receivers_vel.shape[0], velocity.shape[1], velocity.shape[2]))
    #shape: [n_edges, n_timesteps, 2]

    #compute normalized distance histogram
    distance_example = senders_loc-receivers_loc
    distance_example = distance_example**2
    distance_example = distance_example.sum(-1)
    distance_example = np.sqrt(distance_example)

    bins = np.arange(distance_train_min, distance_train_max, 4)

    
    #Normalized histogram of distances
    hist_dist_example = []
    for d in distance_example:
        if np.histogram(d, bins=bins)[0].sum() > 0:
            hist_d = np.histogram(d, bins=bins)[0]/np.histogram(d,bins=bins)[0].sum()
        else:
            hist_d = np.histogram(d, bins=bins)[0]
        hist_dist_example.append(hist_d)
    hist_dist_example = np.array(hist_dist_example) 
    
    
    
    #compute normalized speed differences histogram
    speed_senders = np.sqrt((senders_vel**2).sum(-1))
    speed_receivers = np.sqrt((receivers_vel**2).sum(-1)) #shape: [n_edges, n_timesteps]   
    diff_speed = np.abs(speed_receivers-speed_senders) #shape: [n_edges, n_timeteps]
    
    bins = np.arange(speedDiff_train_min, speedDiff_train_max, 0.5)
    
    #Normalized histogram of speed differences
    hist_diffSpeed_example = []
    for edge in diff_speed:
        if np.histogram(edge, bins=bins)[0].sum() > 0:
            hist = np.histogram(edge, bins=bins)[0]/np.histogram(edge,bins=bins)[0].sum()
        else:
            hist = np.histogram(edge, bins=bins)[0]
        hist_diffSpeed_example.append(hist)
    hist_diffSpeed_example = np.array(hist_diffSpeed_example)
    
    
    
    #compute normalized velocity
    tans_senders_vel = senders_vel[:,:,1]/senders_vel[:,:,0]
    direc_senders_vel = np.arctan(tans_senders_vel)
    tans_receivers_vel = receivers_vel[:,:,1]/receivers_vel[:,:,0]
    direc_receivers_vel = np.arctan(tans_receivers_vel)
    
    diff_direcVel = np.abs(direc_senders_vel-direc_receivers_vel)
    #shape: [n_edges, n_timesteps]
    
    bins = np.arange(0., np.pi+0.25*np.pi, 0.25*np.pi)
    
    #Normalized histogram of velocity direction difference
    hist_diffVel_example = []
    for edge in diff_direcVel:
        if np.histogram(edge, bins=bins)[0].sum() > 0:
            hist = np.histogram(edge, bins=bins)[0]/np.histogram(edge,bins=bins)[0].sum()
        else:
            hist = np.histogram(edge, bins=bins)[0]
        hist_diffVel_example.append(hist)
    hist_diffVel_example = np.array(hist_diffVel_example)
    
    
    
    #compute normalized velocity/position
    diff_locations = senders_loc-receivers_loc #shape: [n_edges, n_timesteps, 2]
    diff_locations_tans = diff_locations[:,:,1]/diff_locations[:,:,0] #shape: [n_edges, n_timesteps]
    relative_positions = np.arctan(diff_locations_tans) #shape: [n_edges, n_timesteps]
    
    diff_velocities = senders_vel-receivers_vel #shape: [n_edges, n_timesteps, 2]
    diff_velocities_tans = diff_velocities[:,:,1]/diff_velocities[:,:,0] #shape: [n_edges, n_timesteps]
    relative_vel = np.arctan(diff_velocities_tans) #shape: [n_edges, n_timesteps]
    
    diff_vel_loc = np.abs(relative_positions-relative_vel) #shape: [n_edges, n_timesteps]
    
    bins = np.arange(0, np.pi+0.25*np.pi,0.25*np.pi)
    
    #Normalized histogram of VP difference
    hist_diffVP_example = []
    for edge in diff_vel_loc:
        if np.histogram(edge, bins=bins)[0].sum() > 0:
            hist = np.histogram(edge, bins=bins)[0]/np.histogram(edge,bins=bins)[0].sum()
        else:
            hist = np.histogram(edge, bins=bins)[0]
        hist_diffVP_example.append(hist)
    hist_diffVP_example = np.array(hist_diffVP_example)
    
    
    hist_feat_example = np.concatenate([hist_dist_example, hist_diffSpeed_example, 
                                    hist_diffVel_example, hist_diffVP_example], axis=-1)
    
    
    predicted_edges = clf.predict(hist_feat_example)
    predicted_edges[predicted_edges==-1]=0  
    predicted_edges_all.append(predicted_edges)
    predicted_edges_diag = np.diag(predicted_edges)
    predicted_relation = np.matmul(rel_send.T, np.matmul(predicted_edges_diag, rel_rec))
    if predicted_relation.sum()==0:
        pred_con = np.arange(n_atoms)
    else:
        pred_con = get_connected_components(predicted_relation)
        
    label_edges_diag = np.diag(label)
    label_relation = np.matmul(rel_send.T, np.matmul(label_edges_diag, rel_rec))
    if label_relation.sum()==0:
        label_con = np.arange(n_atoms)
    else:
        label_con = get_connected_components(label_relation)
      
    recall, precision, F1 = compute_groupMitre_labels(label_con, pred_con)
    recall_test.append(recall)
    precision_test.append(precision)
    F1_test.append(F1)
    
print("Average recall: ", np.mean(recall_test))
print("Average precision: ", np.mean(precision_test))
print("Average F1: ", np.mean(F1_test))




/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/595932923.py:74: RuntimeWarning: invalid value encountered in true_divide
  tans_senders_vel = senders_vel[:,:,1]/senders_vel[:,:,0]
/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/595932923.py:76: RuntimeWarning: invalid value encountered in true_divide
  tans_receivers_vel = receivers_vel[:,:,1]/receivers_vel[:,:,0]
/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/595932923.py:102: RuntimeWarning: invalid value encountered in true_divide
  diff_velocities_tans = diff_velocities[:,:,1]/diff_velocities[:,:,0] #shape: [n_edges, n_timesteps]
/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/595932923.py:74: RuntimeWarning: invalid value encountered in true_divide
  tans_senders_vel = senders_vel[:,:,1]/senders_vel[:,:,0]
/var/folders/js/md3h5nb949j7585pm47mjshh0000gn/T/ipykernel_55386/595932923.py:76: RuntimeWarning: invalid value encountered in true_divide
  tans_receivers_ve

Average recall:  0.85
Average precision:  0.85
Average F1:  0.85


## Compute edge recall

In [1139]:
def compute_confusion(preds, target):
    """compute TN, FP, FN, TP"""
    #total_positive = ((target[target==1]).sum())
    #total_negative = ((target[target==0]==0).sum())
    true_positive = ((preds[target==1]==1).sum())
    false_negative = ((preds[target==1]==0).sum())
    true_negative = ((preds[target==0]==0).sum())
    false_positive = ((preds[target==0]==1).sum())
    
    #true_positive = ((target[preds==1]==1).sum())
    #true_negative = ((target[preds==0]==0).sum())
    
    #false_positive = total_negative-true_negative
    #false_negative = total_positive-true_positive
    
    #print("total positive: ", total_positive)
    #print("total negative: ", total_negative)
    
    return true_negative, false_negative, false_positive, true_positive

In [1140]:
preds = np.concatenate(predicted_edges_all)
preds
preds.sum()

58

In [1141]:
labels_test[10].shape

(42,)

In [1142]:
labels_test_all = np.concatenate(labels_test)
labels_test_all
labels_test_all.sum()

34

In [1143]:
labels_test_all

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [1144]:
tn, fn, fp, tp = compute_confusion(preds, labels_test_all)
print("True negative: ", tn)
print("False negative: ", fn)
print("False positive: ", fp)
print("True positive: ", tp)


print("normalized tn: ", tn/(tn+fp))
print("normalized fn: ", fn/(tp+fn))
print("normalized fp: ", fp/(fp+tn))
print("normalized tp: ", tp/(fn+tp))

True negative:  216
False negative:  0
False positive:  24
True positive:  34
normalized tn:  0.9
normalized fn:  0.0
normalized fp:  0.1
normalized tp:  1.0
